In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns

from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly, Stations

In [ ]:
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

In [ ]:
df = pd.read_csv("C:/Users/cmgas/Downloads/archive (2)/US_Accidents_Dec21_updated.csv")
df.head()

In [ ]:
#impute the wind chill column
df['Wind_Chill(F)'] = df['Wind_Chill(F)'].fillna(df['Temperature(F)'])

In [ ]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'])

In [ ]:
#impute the precipitation column

#define a function to calculate the season
def get_season(month, day):
    if (month >= 3 and month <= 5) or (month == 2 and day >= 29):
        return 'spring'
    elif month >= 6 and month <= 8:
        return 'summer'
    elif month >= 9 and month <= 11:
        return 'fall'
    else:
        return 'winter'

# Apply get_season function to create new column
df['season'] = df['Start_Time'].apply(lambda x: get_season(x.month, x.day))

In [ ]:
#calculate average precipitation when it is precipitating in a given state for a given season

#define a function to calculate seasonal average precipitation
def seasonal_avg_precip(group):
    # Filter for non-zero precipitation
    group = group[group['Precipitation(in)'] != 0]
    # Calculate mean precipitation
    avg_precip = group['Precipitation(in)'].mean()
    return avg_precip

#group the dataframe by state and season, and calculate seasonal average precipitation
seasonal_avg = df.groupby(['State', 'season']).apply(seasonal_avg_precip).reset_index(name='Seasonal_Avg_Precip')

seasonal_avg.head()

In [ ]:
df = pd.merge(df, seasonal_avg, on=['State', 'season'], how='left')

weather_not_null = df[df['Weather_Condition'].notnull()]

# define the list of keywords
keywords = ['Sleet', 'Rain', 'Snow', 'Drizzle', 'Wintry Mix', 'Ice Pellets', 'Hail', 'Showers', 'Squalls']

# create a boolean mask for rows with null precipitation and keywords in weather condition
mask_keywords = weather_not_null['Precipitation(in)'].isnull() & weather_not_null['Weather_Condition'].str.contains('|'.join(keywords))

# impute seasonal average precipitation values where the keyword mask is True
weather_not_null.loc[mask_keywords, 'Precipitation(in)'] = weather_not_null['Seasonal_Avg_Precip']

# create a boolean mask for rows with null precipitation and no keywords in weather condition
mask_no_keywords = weather_not_null['Precipitation(in)'].isnull() & (~weather_not_null['Weather_Condition'].str.contains('|'.join(keywords)))

# impute 0 values where the no keyword mask is True
weather_not_null.loc[mask_no_keywords, 'Precipitation(in)'] = 0

weather_not_null['Precipitation(in)'].isna().sum()

In [ ]:
df.update(weather_not_null[['Precipitation(in)']])

In [ ]:
#use meteostat to try to impute more missing values

#get hourly weather data from meteostat using x-y coordinates
def get_hourly_weather(latitude, longitude, timestamp):
    # Find the closest weather station to the location
    stations = Stations()
    station = stations.nearby(latitude, longitude).fetch(1)

    #check if station was found
    if not station.empty:
        station_id = station.index[0]

        #extract the hour from timestamp
        hour = timestamp.hour

        #calculate start and end time for hourly data with the specific hour
        start_time = timestamp.replace(hour=hour) - timedelta(hours=1)
        end_time = timestamp.replace(hour=hour) + timedelta(hours=1)

        # Fetch hourly data for the location and time
        hourly_data = Hourly(station_id, start_time, end_time)
        hourly_data = hourly_data.fetch()

        hourly_data.index = pd.to_datetime(hourly_data.index)

        # Filter hourly data to only include data for the specified hour
        hourly_data = hourly_data[hourly_data.index.hour == hour]

        #convert temperature from C to F
        hourly_data['temp'] = (hourly_data['temp'] * 9/5) + 32

        #convert pressure from hPA to inches
        hourly_data['pres'] = hourly_data['pres'] * 0.02953

        #convert precipitation from mm to inches
        hourly_data['prcp'] = hourly_data['prcp'] / 25.4 

        #convert wind speed from km/hr to mi/hr
        hourly_data['wspd'] = hourly_data['wspd'] * 0.621371

        return hourly_data
    else:
        return None

In [ ]:
#test the function
test_data = {'Latitude': [39.865420],
             'Longitude': [-84.062800],
             'Timestamp': [datetime(2016, 2, 8, 5, 56, 20)]}
test_df = pd.DataFrame(test_data)

test_weather = get_hourly_weather(test_df['Latitude'].iloc[0], test_df['Longitude'].iloc[0], test_df['Timestamp'].iloc[0])
print(test_weather)

In [ ]:
columns_to_fetch = ['rhum', 'temp', 'prcp', 'wspd', 'pres']
column_names = ['Humidity(%)', 'Temperature(F)', 'Precipitation(in)', 'Wind_Speed(mph)', 'Pressure(in)']

In [ ]:
subset = df[df[column_names].isnull().any(axis=1)]

In [ ]:
# Iterate over rows in the df that have null values in any of the columns to fetch
counter = 0
for index, row in subset[subset[column_names].isnull().any(axis=1)].iterrows():
    latitude = row['Start_Lat']
    longitude = row['Start_Lng']
    timestamp = row['Start_Time']

    try:
      weather_data = get_hourly_weather(latitude, longitude, timestamp)
    except AttributeError:
      print(f"Error: Index {index} has no attribute 'hour'")
      continue

    if weather_data is not None:
        for i, column in enumerate(columns_to_fetch):
          if pd.isnull(row[column_names[i]]):
            if not weather_data[column].empty:
              subset.loc[index, column_names[i]] = weather_data[column].values[0]
              print('success - weather added')
            else:
              continue
    
    counter += 1
print(f'Total iterations: {counter}')

In [ ]:
df.update(subset)

In [ ]:
#drop rows where everything is still null
cols_to_check = ['Wind_Speed(mph)', 'Visibility(mi)', 'Weather_Condition', 'Humidity(%)', 'Temperature(F)', 'Pressure(in)', 'Precipitation(in)']
final_df = df.dropna(subset=cols_to_check, how='all')

In [ ]:
#test regression to impute missing weather values

cols_to_check = ['Wind_Speed(mph)', 'Visibility(mi)', 'Weather_Condition', 'Humidity(%)', 'Temperature(F)', 'Wind_Chill(F)', 'Pressure(in)', 'Precipitation(in)']


null_percent = final_df[cols_to_check].isnull().mean() * 100
print(null_percent)

In [ ]:
# less than 5% NAs in remaining rows - imputation via regression is valid

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [ ]:
#try regression for wind speed as the column with the most remaining nulls

df_missing = df[df['Wind_Speed(mph)'].isnull()]
df_not_missing = df[~df['Wind_Speed(mph)'].isnull()]

predictor_cols = ['Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Precipitation(in)']

X = df_not_missing[predictor_cols]
y = df_not_missing['Wind_Speed(mph)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'tree_method': 'exact',
    'enable_categorical': True
}

num_round = 100
bst = xgb.train(params, dtrain, num_round)

In [ ]:
y_pred = bst.predict(dtest)

# Calculate accuracy score
from sklearn.metrics import r2_score
accuracy = r2_score(y_test, y_pred)
print("Accuracy score on test data:", accuracy)

In [ ]:
#that R2 score was terrible; let's see if we can build a better regression model to impute nulls in visibility

df_for_visibility = df.copy()
df_for_visibility['Start_Time'] = pd.to_datetime(df_for_visibility['Start_Time'])
df_for_visibility['Start_Hour'] = df_for_visibility['Start_Time'].apply(lambda x: x.hour)
df_for_visibility['Weather_Condition'] = df_for_visibility['Weather_Condition'].fillna('')  # fill NaN values with empty string


In [ ]:
rain_keywords = ['Rain', 'Drizzle', 'Showers', 'Squalls']

# create a function to check if any keyword is present in the string
def check_keywords(row):
    for keyword in rain_keywords:
        if keyword in str(row['Weather_Condition']):
            return 1
    return 0

# create the 'Rain' binary predictor column
df_for_visibility['Rain'] = df_for_visibility.apply(check_keywords, axis=1)

snow_keywords = ['Sleet', 'Snow', 'Wintry Mix', 'Ice Pellets', 'Hail']

# create a function to check if any keyword is present in the string
def check_keywords(row):
    for keyword in snow_keywords:
        if keyword in str(row['Weather_Condition']):
            return 1
    return 0

# create the 'Snow' binary predictor column
df_for_visibility['Snow'] = df_for_visibility.apply(check_keywords, axis=1)

In [ ]:
df_missing2 = df_for_visibility[final_df['Visibility(mi)'].isnull()]
df_not_missing2 = df_for_visibility[~final_df['Visibility(mi)'].isnull()]

In [ ]:
predictor_cols2 = ['Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Precipitation(in)', 'Start_Hour', 'Rain', 'Snow']

X1 = df_not_missing2[predictor_cols2]
y1 = df_not_missing2['Visibility(mi)']

In [ ]:
# Split the data into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

dtrain1 = xgb.DMatrix(X_train1, label=y_train1)
dtest1 = xgb.DMatrix(X_test1, label=y_test1)

params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'tree_method': 'exact',
}

num_round = 100
visib_model = xgb.train(params, dtrain1, num_round)

In [ ]:
y_pred1 = visib_model.predict(dtest1)

# Calculate accuracy score
accuracy1 = r2_score(y_test1, y_pred1)
print("Accuracy score on test data:", accuracy1)

In [ ]:
#this regression model is also terrible, so we won't impute using regression